In [1]:
import pandas as pd
import glob
import numpy as np
import itertools

In [2]:

researchers = glob.glob("data/doras/*.csv")
doras_df = pd.DataFrame(columns=['Publication Title','Uncontrolled Keywords'])
doras_df
for researcher_doras in researchers:
    researcher_df = pd.read_csv(researcher_doras)[['Publication Title','Uncontrolled Keywords']]
    doras_df = pd.concat([doras_df,researcher_df],ignore_index=True)


In [3]:

doras_df = doras_df.drop_duplicates(subset=['Publication Title'])
keywords = set()
for keyword in doras_df["Uncontrolled Keywords"]:
    if str(keyword) != 'nan':
        for category in keyword.split(';'):
            category = category.strip()
            if category:
                keywords.add(category)
pairs_keywords = list(itertools.combinations(keywords, 2))


In [4]:
keyword_1_list = []
keyword_2_list = []
pair_count_list = []
for keyword_1, keyword_2 in pairs_keywords:
    pair_count = 0
    for keyword in  doras_df["Uncontrolled Keywords"]:
        if keyword_1 in str(keyword) and keyword_2 in str(keyword):
            pair_count += 1
    if pair_count > 0:
        keyword_1_list.append(keyword_1)
        keyword_2_list.append(keyword_2)
        pair_count_list.append(pair_count)


In [6]:

doras_keyword_df = pd.DataFrame(data={'keyword_1':keyword_1_list,'keyword_2':keyword_2_list,'count':pair_count_list})
doras_keyword_df.to_csv('graph-anayltics/cleaned_data/doras_keywords.csv', index=False)

keyword_vertices_name = sorted(set(doras_keyword_df['keyword_1'].values).union(set(doras_keyword_df['keyword_2'].values)))

doras_vertices = pd.DataFrame(data={'id':range(1,len(keyword_vertices_name)+1),'keyword':keyword_vertices_name})
doras_vertices.to_csv('graph-anayltics/cleaned_data/doras_vertices.csv', index=False)
doras_vertices_dict = dict(zip(keyword_vertices_name, range(1,len(keyword_vertices_name)+1)))
for index, row in doras_keyword_df.iterrows():
    keyword_1_id = doras_vertices_dict[row['keyword_1']]
    keyword_2_id = doras_vertices_dict[row['keyword_2']]
    if keyword_1_id < keyword_2_id:
        doras_keyword_df.iloc[index, 0] = keyword_1_id
        doras_keyword_df.iloc[index, 1] = keyword_2_id
    if keyword_1_id > keyword_2_id:
        doras_keyword_df.iloc[index, 1] = keyword_1_id
        doras_keyword_df.iloc[index, 0] = keyword_2_id

doras_keyword_df

doras_keyword_df.describe()
doras_keyword_df.to_csv('graph-anayltics/cleaned_data/doras_edges.csv', index=False)
